In [1]:
import glob
import os
import re
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

# Paths
DIRECTORY = "/home/user/datasets/resized_train_cropped/resized_train_cropped"
LABELS_FILE = "/home/user/datasets/trainLabels_cropped.csv"

# Load labels
labels_df = pd.read_csv(LABELS_FILE)
labels_dict = dict(zip(labels_df['image'], labels_df['level']))

# Collect all images in all subfolders
all_files = glob.glob(os.path.join(DIRECTORY, "**", "*.jpeg"), recursive=True)

# ===== PARAMÈTRES FIXES POUR L'ENTRAÎNEMENT =====
START_INDEX = 0      # Toujours commencer à 0
BATCH_SIZE = 10000    # Toujours charger 1000 images
# ===============================================

images_train = []
labels_train = []
current_index = 0
loaded_count = 0

for filepath in all_files:
    filename = os.path.basename(filepath)

    # Charger seulement BATCH_SIZE images
    if loaded_count >= BATCH_SIZE:
        break

    img = load_img(filepath, target_size=(256, 256))
    img_array = img_to_array(img) / 255.0

    # Match labels
    base_name = re.sub(r'\s*\(.*\)', '', filename.split('.')[0])
    label = labels_dict.get(base_name)

    if label is not None:
        images_train.append(img_array)
        labels_train.append(label)
        loaded_count += 1
        current_index += 1
    else:
        current_index += 1

# Convert to arrays
images_train = np.array(images_train)
labels_train_raw = np.array(labels_train)

# One-hot encoding
labels_train = to_categorical(labels_train, num_classes=5)

print(f"✅ Entraînement: Chargé {images_train.shape[0]} images (indices 0 à {loaded_count - 1})")
print(f"   Distribution des classes: {np.bincount(labels_train_raw)}")

2025-11-18 11:55:28.038422: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


✅ Entraînement: Chargé 10000 images (indices 0 à 9999)
   Distribution des classes: [7366  721 1475  247  191]


In [2]:
import glob
import os
import re
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

# Paths
DIRECTORY = "/home/user/datasets/resized_train_cropped/resized_train_cropped"
LABELS_FILE = "/home/user/datasets/trainLabels_cropped.csv"

# Load labels
labels_df = pd.read_csv(LABELS_FILE)
labels_dict = dict(zip(labels_df['image'], labels_df['level']))

# Collect all images in all subfolders
all_files = glob.glob(os.path.join(DIRECTORY, "**", "*.jpeg"), recursive=True)

# ===== PARAMÈTRES VARIABLES POUR LE TEST =====
START_INDEX = 20000   # 🔧 MODIFIEZ ICI: où commencer (ex: 1000, 2000, etc.)
BATCH_SIZE = 1       # 🔧 MODIFIEZ ICI: combien d'images (ex: 1, 5, 10, etc.)
# ============================================

images_test = []
labels_test = []
current_index = 0
loaded_count = 0

for filepath in all_files:
    filename = os.path.basename(filepath)

    # Skip jusqu'à START_INDEX
    if current_index < START_INDEX:
        base_name = re.sub(r'\s*\(.*\)', '', filename.split('.')[0])
        if labels_dict.get(base_name) is not None:
            current_index += 1
        continue

    # Charger seulement BATCH_SIZE images
    if loaded_count >= BATCH_SIZE:
        break

    img = load_img(filepath, target_size=(256, 256))
    img_array = img_to_array(img) / 255.0

    # Match labels
    base_name = re.sub(r'\s*\(.*\)', '', filename.split('.')[0])
    label = labels_dict.get(base_name)

    if label is not None:
        images_test.append(img_array)
        labels_test.append(label)
        loaded_count += 1
        current_index += 1
    else:
        current_index += 1

# Convert to arrays
images_test = np.array(images_test)
labels_test_raw = np.array(labels_test)

# One-hot encoding
labels_test = to_categorical(labels_test, num_classes=5)

print(f"✅ Test: Chargé {images_test.shape[0]} images (indices {START_INDEX} à {START_INDEX + loaded_count - 1})")
print(f"   Classes: {labels_test_raw}")

✅ Test: Chargé 1 images (indices 20000 à 20000)
   Classes: [0]


In [3]:
x_train = images_train.reshape((images_train.shape[0], -1))  # Shape: (n_train, 256*256*3)
y_train = np.argmax(labels_train, axis=1)

In [4]:
x_test = images_test.reshape((images_test.shape[0], -1))  # Shape: (n_train, 256*256*3)
y_test = np.argmax(labels_test, axis=1)

In [10]:
import numpy as np
from collections import Counter

class knn:
    def __init__(self, k):
        self.k = k

    def fitt(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    def predire(self, test_sample):
        #test_sample = np.array(test_sample).ravel()
        # Vectorized Euclidean distance: sqrt(sum((x_train - test_sample)^2))
        distances = np.sqrt(np.sum((self.x_train - test_sample)**2, axis=1))
        print(f'thread {threading.get_ident()} calcule {len(distances)} distances')
        # Get indices of k smallest distances
        k_indices = np.argpartition(distances, self.k)[:self.k]

        # Get the labels of those k neighbors
        k_nearest_labels = self.y_train[k_indices]

        # Return the most common label
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

In [11]:
knn_seq = knn(k=5)
knn_seq.fitt(x_train, y_train)

In [12]:
import time
import threading
predictions = []
debut = time.perf_counter()
for i in x_test:
  prediction = knn_seq.predire(i)
  predictions.append(prediction)
fin = time.perf_counter()
print(f'Temps de prédiction = {fin-debut:.4f} secondes')
print(f"Predicted labels: {predictions}")
print(f"True labels: {labels_test_raw}")  # ✅ Correction: labels_test_raw au lieu de y_test

# ✅ Correction: comparaison correcte
correct = all(pred == true for pred, true in zip(predictions, labels_test_raw))
print(f"Correct? {'✅' if correct else '❌'}")

# Afficher les détails par image
print("\nDétails:")
for idx, (pred, true) in enumerate(zip(predictions, labels_test_raw)):
    match = "✅" if pred == true else "❌"
    print(f"  Image {idx}: Prédiction={pred}, Vrai={true} {match}")

thread 126979215381568 calcule 10000 distances
Temps de prédiction = 10.7195 secondes
Predicted labels: [np.int64(0)]
True labels: [0]
Correct? ✅

Détails:
  Image 0: Prédiction=0, Vrai=0 ✅


In [13]:
import numpy as np
from collections import Counter
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors import ThreadPoolExecutor

from parsl.providers import LocalProvider

@python_app
def calculate_distances_chunk(x_chunk, y_chunk, test_sample, k):
    """
    Parsl app: Calculate distances for a data chunk.
    Returns k nearest distances and labels from this chunk.
    """
    import numpy as np
    
    # Calculate squared Euclidean distances (skip sqrt for efficiency)
    distances = np.sum((x_chunk - test_sample)**2, axis=1)
    
    # Get k smallest distances from this chunk
    chunk_k = min(k, len(distances))
    k_indices = np.argpartition(distances, chunk_k - 1)[:chunk_k]
    
    # Return distances and corresponding labels
    return distances[k_indices], y_chunk[k_indices]


class ParslKNN:
    def __init__(self, k, n_workers=4):
        """
        Initialize Parsl KNN classifier.
        
        Args:
            k: Number of nearest neighbors
            n_workers: Number of parallel workers
        """
        self.k = k
        self.n_workers = n_workers
        self.parsl_loaded = False
        

    def fitt(self, x_train, y_train):
        """
        Fit the KNN model with training data.
        
        Args:
            x_train: Training features (n_samples, n_features)
            y_train: Training labels (n_samples,)
        """
        
        
        self.x_train = np.asarray(x_train)
        self.y_train = np.asarray(y_train)
        self.n_samples = len(self.x_train)
        
        # Pre-split data into chunks for workers
        chunk_size = (self.n_samples + self.n_workers - 1) // self.n_workers
        self.data_chunks = []
        
        for i in range(self.n_workers):
            start_idx = i * chunk_size
            end_idx = min(start_idx + chunk_size, self.n_samples)
            
            if start_idx < self.n_samples:
                self.data_chunks.append((
                    self.x_train[start_idx:end_idx],
                    self.y_train[start_idx:end_idx]
                ))
        
        print(f"Data split into {len(self.data_chunks)} chunks for {self.n_workers} workers")
    
    def predire(self, test_sample):
        """
        Predict the class label for a test sample using parallel computation.
        
        Args:
            test_sample: Test sample features (n_features,)
            
        Returns:
            Predicted class label
        """
        if not hasattr(self, 'data_chunks'):
            raise RuntimeError("Model not fitted. Call fitt() first.")
        
        test_sample = np.asarray(test_sample)
        
        # Submit parallel tasks to compute distances for each chunk
        futures = []
        for x_chunk, y_chunk in self.data_chunks:
            future = calculate_distances_chunk(x_chunk, y_chunk, test_sample, self.k)
            futures.append(future)
        
        # Wait for all workers to complete and collect results
        chunk_results = [future.result() for future in futures]
        
        # Merge all k*workers nearest neighbors
        all_distances = np.concatenate([result[0] for result in chunk_results])
        all_labels = np.concatenate([result[1] for result in chunk_results])
        
        # Select final k nearest neighbors from merged results
        final_k = min(self.k, len(all_distances))
        final_k_indices = np.argpartition(all_distances, final_k - 1)[:final_k]
        final_k_labels = all_labels[final_k_indices]
        
        # Return most common label (majority vote)
        most_common = Counter(final_k_labels).most_common(1)
        return most_common[0][0]
    
    def predire_batch(self, test_samples):
        """
        Predict labels for multiple test samples.
        
        Args:
            test_samples: Array of test samples (n_test_samples, n_features)
            
        Returns:
            List of predicted labels
        """
        return [self.predire(sample) for sample in test_samples]
    
    def cleanup(self):
        """Clean up Parsl resources."""
        if self.parsl_loaded:
            parsl.dfk().cleanup()
            parsl.clear()
            self.parsl_loaded = False

In [18]:
config = Config(
                executors=[
                    ThreadPoolExecutor(
                        max_threads=12,
                        label='knn_workers'
                    )
                ],
                strategy='none',  # Disable auto-scaling for predictable performance
            )
parsl.load(config)


In [19]:
import time
knn_parsl = ParslKNN(k=5, n_workers=12)
knn_parsl.fitt(x_train, y_train)

debut = time.time()
prediction = knn_parsl.predire(x_test)
fin = time.time()
print(f"Predicted class: {prediction == y_test}")
print(f'prediction time: {fin-debut:.2f} secondes')

Data split into 12 chunks for 12 workers
Predicted class: [ True]
prediction time: 2.04 secondes


In [17]:
parsl.clear()

ray

In [46]:
import numpy as np
from collections import Counter
import ray
import time

@ray.remote
def calculate_distances_chunk(x_chunk, y_chunk, test_sample, k):
    """
    Ray remote function: Calculate distances for a data chunk.
    Returns k nearest distances and labels from this chunk.
    """
    import numpy as np
    
    # Calculate squared Euclidean distances (skip sqrt for efficiency)
    distances = np.sum((x_chunk - test_sample)**2, axis=1)
    
    # Get k smallest distances from this chunk
    chunk_k = min(k, len(distances))
    k_indices = np.argpartition(distances, chunk_k - 1)[:chunk_k]
    
    # Return distances and corresponding labels
    return distances[k_indices], y_chunk[k_indices]

class RayKNN:
    def __init__(self, k, n_workers=4, verbose=True):
        """
        Initialize Ray KNN classifier.
        
        Args:
            k: Number of nearest neighbors
            n_workers: Number of parallel workers
            verbose: Whether to print timing information
        """
        self.k = k
        self.n_workers = n_workers
        self.verbose = verbose
        
    def fitt(self, x_train, y_train):
        """
        Fit the KNN model with training data.
        
        Args:
            x_train: Training features (n_samples, n_features)
            y_train: Training labels (n_samples,)
        """
        self.x_train = np.asarray(x_train)
        self.y_train = np.asarray(y_train)
        self.n_samples = len(self.x_train)
        
        # Pre-split data into chunks for workers
        chunk_size = (self.n_samples + self.n_workers - 1) // self.n_workers
        self.data_chunks = []
        
        for i in range(self.n_workers):
            start_idx = i * chunk_size
            end_idx = min(start_idx + chunk_size, self.n_samples)
            
            if start_idx < self.n_samples:
                self.data_chunks.append((
                    self.x_train[start_idx:end_idx],
                    self.y_train[start_idx:end_idx]
                ))
        
        print(f"Data split into {len(self.data_chunks)} chunks for {self.n_workers} workers")
    
    def predire(self, test_sample):
        """
        Predict the class label for a test sample using parallel computation.
        
        Args:
            test_sample: Test sample features (n_features,)
            
        Returns:
            Predicted class label
        """
        overall_start = time.time()
        
        if not hasattr(self, 'data_chunks'):
            raise RuntimeError("Model not fitted. Call fitt() first.")
        
        # Convert test sample
        convert_start = time.time()
        test_sample = np.asarray(test_sample)
        convert_time = time.time() - convert_start
        
        # Submit parallel tasks to compute distances for each chunk
        submit_start = time.time()
        futures = []
        for x_chunk, y_chunk in self.data_chunks:
            future = calculate_distances_chunk.remote(x_chunk, y_chunk, test_sample, self.k)
            futures.append(future)
        submit_time = time.time() - submit_start
        
        # Wait for all workers to complete and collect results
        wait_start = time.time()
        chunk_results = ray.get(futures)
        wait_time = time.time() - wait_start
        
        # Merge all k*workers nearest neighbors
        merge_start = time.time()
        all_distances = np.concatenate([result[0] for result in chunk_results])
        all_labels = np.concatenate([result[1] for result in chunk_results])
        merge_time = time.time() - merge_start
        
        # Select final k nearest neighbors from merged results
        final_select_start = time.time()
        final_k = min(self.k, len(all_distances))
        final_k_indices = np.argpartition(all_distances, final_k - 1)[:final_k]
        final_k_labels = all_labels[final_k_indices]
        final_select_time = time.time() - final_select_start
        
        # Return most common label (majority vote)
        vote_start = time.time()
        most_common = Counter(final_k_labels).most_common(1)
        vote_time = time.time() - vote_start
        
        total_time = time.time() - overall_start
        
        if self.verbose:
            print(f"\n=== PREDIRE TIMING (single sample) ===")
            print(f"Test sample conversion: {convert_time:.4f}s")
            print(f"Task submission: {submit_time:.4f}s")
            print(f"Worker execution (wait): {wait_time:.4f}s")
            print(f"Result merging: {merge_time:.4f}s")
            print(f"Final k selection: {final_select_time:.4f}s")
            print(f"Majority voting: {vote_time:.4f}s")
            print(f"Total predire time: {total_time:.4f}s")
        
        return most_common[0][0]
    
    def predire_batch(self, test_samples):
        """
        Predict labels for multiple test samples.
        
        Args:
            test_samples: Array of test samples (n_test_samples, n_features)
            
        Returns:
            List of predicted labels
        """
        return [self.predire(sample) for sample in test_samples]

In [50]:
import time
ray.init(num_cpus=12)
print(ray.cluster_resources())
knn = RayKNN(5, 12)
knn.fitt(x_train, y_train)
debut = time.time()
prediction = knn.predire(x_test)
fin = time.time()
print(f'prediction time = {fin-debut:.4f} secondes')
prediction == y_test
ray.shutdown()

2025-11-18 11:30:21,852	INFO worker.py:2013 -- Started a local Ray instance.


{'CPU': 12.0, 'node:__internal_head__': 1.0, 'memory': 15262773658.0, 'object_store_memory': 6541188710.0, 'node:192.168.62.206': 1.0}
Data split into 12 chunks for 12 workers

=== PREDIRE TIMING (single sample) ===
Test sample conversion: 0.0000s
Task submission: 2.9503s
Worker execution (wait): 1.0277s
Result merging: 0.0001s
Final k selection: 0.0001s
Majority voting: 0.0001s
Total predire time: 3.9783s
prediction time = 3.9801 secondes


In [45]:
ray.shutdown()

dask

In [13]:
import numpy as np
from collections import Counter
import time
from dask.distributed import Client

def calculate_distances_chunk(x_chunk, y_chunk, test_sample, k):
    """
    Dask function: Calculate distances for a data chunk.
    Returns k nearest distances and labels from this chunk.
    """
    import numpy as np
    
    # Calculate squared Euclidean distances (skip sqrt for efficiency)
    distances = np.sum((x_chunk - test_sample)**2, axis=1)
    
    # Get k smallest distances from this chunk
    chunk_k = min(k, len(distances))
    k_indices = np.argpartition(distances, chunk_k - 1)[:chunk_k]
    
    # Return distances and corresponding labels
    return distances[k_indices], y_chunk[k_indices]


def calculate_distances_wrapper(chunk_tuple, test_sample, k):
    """
    Wrapper function to unpack chunk tuple for Dask submit.
    """
    x_chunk, y_chunk = chunk_tuple
    return calculate_distances_chunk(x_chunk, y_chunk, test_sample, k)


class DaskKNN:
    def __init__(self, k, client, verbose=True):
        """
        Initialize Dask KNN classifier.
        
        Args:
            k: Number of nearest neighbors
            client: Dask Client instance (required)
            verbose: Whether to print timing information
        """
        self.k = k
        self.client = client
        self.verbose = verbose
        self.scattered_chunks = None
        
    def fitt(self, x_train, y_train):
        """
        Fit the KNN model with training data.
        
        Args:
            x_train: Training features (n_samples, n_features)
            y_train: Training labels (n_samples,)
        """
        if self.client is None:
            raise RuntimeError("Client not initialized. Pass a Dask Client to __init__().")
        
        self.x_train = np.asarray(x_train)
        self.y_train = np.asarray(y_train)
        self.n_samples = len(self.x_train)
        
        # Pre-split data into chunks for workers
        n_workers = len(self.client.scheduler_info()['workers'])
        chunk_size = (self.n_samples + n_workers - 1) // n_workers
        data_chunks = []
        
        for i in range(n_workers):
            start_idx = i * chunk_size
            end_idx = min(start_idx + chunk_size, self.n_samples)
            
            if start_idx < self.n_samples:
                data_chunks.append((
                    self.x_train[start_idx:end_idx],
                    self.y_train[start_idx:end_idx]
                ))
        
        # Scatter data chunks to workers for reuse
        self.scattered_chunks = self.client.scatter(data_chunks, broadcast=True)
        
        print(f"Data split into {len(data_chunks)} chunks for {n_workers} workers")
    
    def predire(self, test_sample):
        """
        Predict the class label for a test sample using parallel computation.
        
        Args:
            test_sample: Test sample features (n_features,)
            
        Returns:
            Predicted class label
        """
        overall_start = time.time()
        
        if self.scattered_chunks is None:
            raise RuntimeError("Model not fitted. Call fitt() first.")
        
        # Convert test sample
        convert_start = time.time()
        test_sample = np.asarray(test_sample)
        convert_time = time.time() - convert_start
        
        # Submit parallel tasks to compute distances for each chunk
        submit_start = time.time()
        futures = []
        for chunk_future in self.scattered_chunks:
            future = self.client.submit(
                calculate_distances_wrapper,
                chunk_future,
                test_sample,
                self.k
            )
            futures.append(future)
        submit_time = time.time() - submit_start
        
        # Wait for all workers to complete and collect results
        wait_start = time.time()
        chunk_results = self.client.gather(futures)
        wait_time = time.time() - wait_start
        
        # Merge all k*workers nearest neighbors
        merge_start = time.time()
        all_distances = np.concatenate([result[0] for result in chunk_results])
        all_labels = np.concatenate([result[1] for result in chunk_results])
        merge_time = time.time() - merge_start
        
        # Select final k nearest neighbors from merged results
        final_select_start = time.time()
        final_k = min(self.k, len(all_distances))
        final_k_indices = np.argpartition(all_distances, final_k - 1)[:final_k]
        final_k_labels = all_labels[final_k_indices]
        final_select_time = time.time() - final_select_start
        
        # Return most common label (majority vote)
        vote_start = time.time()
        most_common = Counter(final_k_labels).most_common(1)
        vote_time = time.time() - vote_start
        
        total_time = time.time() - overall_start
        
        if self.verbose:
            print(f"\n=== PREDIRE TIMING (single sample) ===")
            print(f"Test sample conversion: {convert_time:.4f}s")
            print(f"Task submission: {submit_time:.4f}s")
            print(f"Worker execution (wait): {wait_time:.4f}s")
            print(f"Result merging: {merge_time:.4f}s")
            print(f"Final k selection: {final_select_time:.4f}s")
            print(f"Majority voting: {vote_time:.4f}s")
            print(f"Total predire time: {total_time:.4f}s")
        
        return most_common[0][0]
    
    def predire_batch(self, test_samples):
        """
        Predict labels for multiple test samples.
        
        Args:
            test_samples: Array of test samples (n_test_samples, n_features)
            
        Returns:
            List of predicted labels
        """
        return [self.predire(sample) for sample in test_samples]

In [14]:
client.close()
print("Starting Dask Client...")
client = Client(n_workers=2, threads_per_worker=1)
print(f"Dask Dashboard: {client.dashboard_link}")

Starting Dask Client...
Dask Dashboard: http://127.0.0.1:8787/status


In [ ]:
import time
knn_dask = DaskKNN(5, client=client)
begin = time.time()
knn_dask.fitt(x_train, y_train)
end = time.time()
print(f'scatter time = {end - begin:.4f} secondes')
debut = time.time()
prediction = knn_dask.predire(x_test)
fin = time.time()
print(f'prediction time = {fin-debut:.4f} secondes')
prediction == y_test